In [1]:
# Auto-reload imports so you don't have to keep restarting the notebook kernel.
%load_ext autoreload
%autoreload complete --print

# Intro to Full Stack: Databases

This applied practice will be focusing mainly on interacting with SQL databases
through the widely used Object-Relational Mapping (ORM) library, SQLAlchemy.

In this notebook, comments are provided for each significant code block to help you understand what it does. At the same time, you should cross-check with the cell's output to see the SQL queries being executed by SQLAlchemy for each code block.

## Miscellaneous

Run through these after starting/restarting the notebook kernel.

In [2]:
from pathlib import Path

cwd = Path.cwd()

# File paths to the template database and the actual database this workshop will use.
TEMPLATE_DB = cwd / "src" / "template.db"
EXAMPLE_DB = cwd / "my_db.db"

### Enable Printing SQL Statements Made by SQLAlchemy

In [ ]:
import logging

# Log SQL statements made by SQLAlchemy, I suggest you turn this off once you get a hang of it.
LOG_SQLALCHEMY = True

logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(
    logging.INFO if LOG_SQLALCHEMY else logging.WARNING
)

print(
    "SQLAlchemy logging is enabled."
    if LOG_SQLALCHEMY
    else "SQLAlchemy logging is disabled."
)

## Looking at Schemas

First, view the `User` database schema in [`src/models/user.py`](src/models/user.py). Take note of the column types and special attributes like `default` and `unique`.

### Create All Tables

Declaring the schema is not enough; We have to instruct SQLAlchemy to
`CREATE TABLE` in the database based on the schemas. This has to be done once for
a fresh database or when new schemas are added.

In [ ]:
from src import Session, Base

# Delete existing database.
try:
    EXAMPLE_DB.unlink(missing_ok=True)
except PermissionError:
    raise RuntimeError(
        "The database file is already in use, restart the notebook if really want to delete it."
    )

# Create all tables.
with Session.begin() as sess:
    Base.metadata.create_all(sess.bind)


### Reset Database File

For this workshop however, I already prepared a database for you. Below copies it over :)

In [ ]:
import shutil

# Overwrite the workshop database with the prepared example database.
shutil.copy(TEMPLATE_DB, EXAMPLE_DB)

## Example: CRUD with Users

First, lets generate some user details to play with.

In [ ]:
from src.utils import random_name, hash_passwd

# Random username.
EXAMPLE_USERNAME = random_name()
# Password hash for "password".
EXAMPLE_PASSWD = hash_passwd("password")
# The user ID will be assigned later.
EXAMPLE_USER_ID = None

print(f"[Username for Example]\n{EXAMPLE_USERNAME}\n")
print(f"[Password for Example]\n{EXAMPLE_PASSWD}")

### Creating a User

Below snippet creates a user with the given details, then adds it to the database.

In [ ]:
from src import Session, User

with Session() as sess:
    # User created from aforementioned random user details.
    user = User(username=EXAMPLE_USERNAME, password=EXAMPLE_PASSWD)

    # User is added to the current database session, but isn't "saved" yet.
    sess.add(user)
    
    # Notice some values are None when the newly created user hasn't been committed?
    print(f"[`user` Before Commit]\n{user}\n\n", flush=True)

    # Commit the session to save the user to the database.
    print("[Committing...]", flush=True)
    sess.commit()
    print("[Committed]", flush=True)

    # The user ID and other default values are now populated.
    print("\n\n[Reading from DB...]", flush=True)
    # Operations that read user properties implicitly read from the database unless
    # already cached in the session.
    value = str(user)
    print(f"\n\n[`user` After Commit]\n{value}", flush=True)

    # Set the user ID for later use in this notebook.
    EXAMPLE_USER_ID = user.id

Notice that until `sess.commit()` is called, the instance of `User` we created isn't populated with default values yet.

Fields with default values like `id` change from None to their actual values only after the commit, even if the field is non-nullable. This means we shouldn't use the new instance until after its committed.

## Reading a User

Below I show you some of the most common "read" operations you will do. For more
kinds, see <https://docs.sqlalchemy.org/en/20/changelog/migration_20.html#id1>

### By ID

Using `EXAMPLE_USER_ID` from above, we can retrieve the user with that ID.

In [ ]:
with Session() as sess:
    # SQLAlchemy `get` uses the schema type & id to query the database.
    print(f"[Query by id]\n{EXAMPLE_USER_ID}\n", flush=True)
    user = sess.get(User, EXAMPLE_USER_ID)
    print(f"\n\n[User retrieved by id]\n{user}")

### Filtering Users By Exact Value

We can also rerieve users by filtering on a specific column value. In this example, we find all users with nickname "HEROBRINE".

In [ ]:
from sqlalchemy import select

with Session() as sess:
    print('[Filter `nickname` by "HEROBRINE"]', flush=True)
    statement = select(User).filter_by(nickname="HEROBRINE")
    users = sess.execute(statement).scalars().all()
    print(f"\n\n[User retrieved by id]\n{users}")


### Filtering Users By Criteria

In this example, we find users created after 2020 and select the first one.

### FYI: _By `index=True` Columns_

You can read more about what indexes are used for here: <https://www.w3schools.com/sql/sql_create_index.asp>. Notice it is exactly the same as filtering. Meaning the index adds a transparent boost to the query speed. However, indexes shouldn't be used for all columns, do you know why?

In [ ]:

with Session() as sess:
    print(f"[Query by username]\n{EXAMPLE_USERNAME}\n", flush=True)
    # `scalar_one()` expects exactly one result, raising `NoResultFound` or `MultipleResultsFound` otherwise.
    sess.execute(select(User).filter_by(username=EXAMPLE_USERNAME)).scalar_one()
    print(f"\n\n[User retrieved by id]\n{user}")

## Updating a User

### After Being Found Above

### Updating Multiple Users

## Deleting a User

## Your Turn: Transactions CRUD

### Writing the Schema

Fill in the `Transaction` model in [`src/models/transaction.py`](src/models/transaction.py).

### Testing Create

### Testing Read

#### Find by ID

#### Filter by Amount

### Testing Update

### Testing Delete
